# Exploratory Analysis for LLM Travel Chatbot

- This notebook contains exploratory analysis to inform the development of a large language model (LLM) chatbot designed to assist users with questions about their personal travel itineraries.  
   
   
---
   
## Objectives
- Extract gmail from dafamtripbot@gmail.com
- Use LLM to extract flight info from text 


In [ ]:
import base64
import json
import os
from datetime import datetime, date, time
from dotenv import load_dotenv
from pathlib import Path
from typing import Literal, List, Optional

from bs4 import BeautifulSoup  
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field



In [ ]:
load_dotenv(dotenv_path=Path.home() / ".env")

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

In [ ]:
class Passenger(BaseModel):
    first_name: str
    last_name: str

class FlightDetails(BaseModel):
    flight_number: str
    airline_name: str
    departure_date: Optional[date] = None
    departure_time: Optional[time] = None
    arrival_date: Optional[datetime] = None
    arrival_time: Optional[str] = None
    origin: str
    destination: str
    passengers: List[Passenger]

class FlightManifest(BaseModel):
    flights: List[FlightDetails]
    
flight_parser = PydanticOutputParser(pydantic_object=FlightManifest)


In [ ]:
def get_gmail_service() -> Resource:
    """
    Create and return an authenticated Gmail API service instance.

    This function handles authentication with the Gmail API using OAuth 2.0. It first checks
    for saved credentials in a local `token.json` file. If valid credentials are found, they
    are used directly. If not, the function initiates an OAuth 2.0 flow to authenticate
    the user and generates new credentials using client information stored in environment
    variables:
        - TRAVELBOT_GMAIL_CLIENT_ID
        - TRAVELBOT_GMAIL_CLIENT_SECRET

    The credentials are then saved to `token.json` for reuse in future runs.

    Returns:
        googleapiclient.discovery.Resource:
            An authorized Gmail API service instance for making Gmail API calls.

    Raises:
        google.auth.exceptions.GoogleAuthError: If authentication fails or credentials
            cannot be refreshed.
        FileNotFoundError: If `token.json` is missing and OAuth flow cannot retrieve credentials.
    """
    creds: Credentials = None
    # Load saved credentials if available
    if Path('token.json').exists():
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)

    # If no valid credentials, run OAuth flow
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # Get credentials from environment variables
            client_config = {
                "installed": {
                    "client_id": os.getenv("TRAVELBOT_GMAIL_CLIENT_ID"),
                    "client_secret": os.getenv("TRAVELBOT_GMAIL_CLIENT_SECRET"),
                    "auth_uri": "https://accounts.google.com/o/oauth2/auth",
                    "token_uri": "https://oauth2.googleapis.com/token",
                    "redirect_uris": ["http://localhost"]
                }
            }

            flow = InstalledAppFlow.from_client_config(client_config, SCOPES)
            creds = flow.run_local_server(port=0)

        # Save the credentials for next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return build('gmail', 'v1', credentials=creds)

In [ ]:
def get_latest_email_id(client: Optional[Resource] = None) -> Optional[str]:
    """
    Retrieve the message ID of the most recent email from the user's Gmail inbox.

    This function uses the Gmail API to fetch the most recent email for the 
    authenticated user. If a Gmail API client is not provided, one will be created 
    using `get_gmail_service()`.

    Args:
        client (Optional[googleapiclient.discovery.Resource]): 
            An authenticated Gmail API client. If not provided, a new client 
            will be created internally.

    Returns:
        Optional[str]: 
            The message ID of the most recent email if available, otherwise `None`.

    Raises:
        googleapiclient.errors.HttpError: 
            If the Gmail API request fails.# create gmail client if not provided
    """
    # create gmail client if not provided
    if not client:
        client = get_gmail_service()
    # Get list of messages
    results = client.users().messages().list(userId='me', maxResults=1).execute()
    messages = results.get('messages', [])

    if not messages:
        print("No messages found.")
        return

    # Get the message details
    msg_id = messages[0]['id']
    return msg_id

 

In [ ]:
def extract_gmail_as_json(service: Resource, message_id: str) -> Dict[str, Optional[str]]:
    """
    Extract email metadata and body content from the Gmail API and return it as JSON-like data.

    This function retrieves an email message by its ID using the Gmail API, 
    extracts key metadata fields (From, To, Date, Subject, Message ID), 
    and extracts the message body as plain text. If only HTML content 
    is available, it is converted to plain text using BeautifulSoup.

    Args:
        service (googleapiclient.discovery.Resource):
            An authenticated Gmail API service client.
        message_id (str):
            The unique Gmail message ID of the email to retrieve.

    Returns:
        Dict[str, Optional[str]]:
            A dictionary containing the email metadata and body content:
            {
                "from": str or None,
                "to": str or None,
                "date": str or None,
                "subject": str or None,
                "message_id": str,
                "body": str or None
            }

    Raises:
        googleapiclient.errors.HttpError:
            If the Gmail API request fails.

    Example:
        >>> service = get_gmail_service()
        >>> email_data = extract_email_as_json(service, "17c6932b2b4f1a2c")
        >>> print(email_data["subject"])
        'Your Flight Itinerary'
    """
    msg: Dict[str, Any] = service.users().messages().get(userId='me', id=message_id, format='full').execute()

    payload = msg.get("payload", {})
    headers = payload.get("headers", [])

    def get_header(name):
        return next((h['value'] for h in headers if h['name'].lower() == name.lower()), None)

    # Extract headers
    email_data = {
        "from": get_header("From"),
        "to": get_header("To"),
        "date": get_header("Date"),
        "subject": get_header("Subject"),
        "message_id": message_id,
        "body": None  # populated below
    }

    # Extract body (prefer text/plain over text/html)
    def extract_body(payload):
        if payload.get("mimeType") == "text/plain":
            return base64.urlsafe_b64decode(payload["body"].get("data", "")).decode("utf-8", errors="ignore")
        elif payload.get("mimeType") == "text/html":
            html = base64.urlsafe_b64decode(payload["body"].get("data", "")).decode("utf-8", errors="ignore")
            return BeautifulSoup(html, "html.parser").get_text()
        elif "parts" in payload:
            for part in payload["parts"]:
                body = extract_body(part)
                if body:
                    return body
        return None

    email_data["body"] = extract_body(payload)

    return email_data

In [ ]:
# Extract flight info from most recent gmail
msg_id = get_latest_email_id()
data = extract_email_as_json(get_gmail_service(), msg_id)
flight_email = data.get('body', '')

In [ ]:
# Uses environment variable to authenticate 
client = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
# Create chain for extracting flight data from email in JSON format
extract_flight = """I am building a trip itinerary for a family vacation. The following text in triple quotes contains flight information from the email confirmation I received from the airline.  There can be multiple passengers and flights in a single email confirmation. Extract the following relevant passenger and flight information.

The following pieces of information should be collected for each passenger that is traveling. Remember, there can be multiple passengers on each flight
- first_name 
- last_name 

The following pieces of information should be collected for each flight. There are usually multiple flights per email. 
- departure_date
- departure_time
- arrival_date
- arrival_time 
- origin
- destination
- flight_number
- airline_name

 ```{email}```

{format_instructions}
"""

extract_flight_prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract structured flight and passenger information from the text and convert it to JSON using ISO 8601 format for all datetime fields."),
    ("human", extract_flight)
])
flight_chain = extract_flight_prompt | client | flight_parser

In [ ]:
# Extract flight info using chain
flight_chainparams = {'email': flight_email,
                      'format_instructions': flight_parser.get_format_instructions()}
flight_response = flight_chain.invoke(flight_chainparams)

In [ ]:
flight_response.dict()